In [13]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ===============================
# COMPLETE INDICATOR LIBRARY
# ===============================

# === TREND INDICATORS ===
def calculate_sma(data, window):
    return data.rolling(window=window).mean()

def calculate_ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    ema_fast = calculate_ema(data, fast_period)
    ema_slow = calculate_ema(data, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = calculate_ema(macd_line, signal_period)
    histogram = macd_line - signal_line
    return {'macd': macd_line, 'signal': signal_line, 'histogram': histogram}

# === MOMENTUM INDICATORS ===
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_bollinger_bands(data, window=20, num_std=2):
    sma = data.rolling(window=window).mean()
    std = data.rolling(window=window).std()
    upper_band = sma + (std * num_std)
    lower_band = sma - (std * num_std)
    return {'upper': upper_band, 'middle': sma, 'lower': lower_band}

# === VOLUME INDICATORS ===
def calculate_volume_sma(volume, window=20):
    return volume.rolling(window=window).mean()

def calculate_on_balance_volume(close, volume):
    obv = pd.Series(index=close.index, dtype=float)
    obv.iloc[0] = volume.iloc[0]
    
    for i in range(1, len(close)):
        if close.iloc[i] > close.iloc[i-1]:
            obv.iloc[i] = obv.iloc[i-1] + volume.iloc[i]
        elif close.iloc[i] < close.iloc[i-1]:
            obv.iloc[i] = obv.iloc[i-1] - volume.iloc[i]
        else:
            obv.iloc[i] = obv.iloc[i-1]
    return obv

def identify_volume_spikes(volume, threshold_multiplier=2.0, window=20):
    avg_volume = calculate_volume_sma(volume, window)
    return volume > (avg_volume * threshold_multiplier)

# === TRADING SIGNALS ===
def generate_trading_signals(data):
    """Complete trading signal generation"""
    signals = pd.DataFrame(index=data.index)
    
    # Calculate all indicators
    signals['sma_20'] = calculate_sma(data['Close'], 20)
    signals['sma_50'] = calculate_sma(data['Close'], 50)
    signals['ema_12'] = calculate_ema(data['Close'], 12)
    signals['rsi'] = calculate_rsi(data['Close'])
    
    macd_data = calculate_macd(data['Close'])
    signals['macd'] = macd_data['macd']
    signals['macd_signal'] = macd_data['signal']
    
    bollinger = calculate_bollinger_bands(data['Close'])
    signals['bb_upper'] = bollinger['upper']
    signals['bb_lower'] = bollinger['lower']
    
    signals['volume_sma'] = calculate_volume_sma(data['Volume'])
    signals['obv'] = calculate_on_balance_volume(data['Close'], data['Volume'])
    signals['volume_spike'] = identify_volume_spikes(data['Volume'])
    
    # Generate buy/sell signals
    signals['ma_signal'] = 0
    signals['rsi_signal'] = 0
    signals['macd_signal_flag'] = 0
    signals['bb_signal'] = 0
    
    # Moving Average Crossover (Golden Cross)
    signals['ma_signal'] = np.where(
        (signals['sma_20'] > signals['sma_50']) & 
        (signals['sma_20'].shift(1) <= signals['sma_50'].shift(1)), 1,
        np.where(
            (signals['sma_20'] < signals['sma_50']) & 
            (signals['sma_20'].shift(1) >= signals['sma_50'].shift(1)), -1, 0
        )
    )
    
    # RSI Oversold/Overbought
    signals['rsi_signal'] = np.where(signals['rsi'] < 30, 1,  # Oversold = Buy
                                   np.where(signals['rsi'] > 70, -1, 0))  # Overbought = Sell
    
    # MACD Signal Line Crossover
    signals['macd_signal_flag'] = np.where(
        (signals['macd'] > signals['macd_signal']) & 
        (signals['macd'].shift(1) <= signals['macd_signal'].shift(1)), 1,
        np.where(
            (signals['macd'] < signals['macd_signal']) & 
            (signals['macd'].shift(1) >= signals['macd_signal'].shift(1)), -1, 0
        )
    )
    
    # Bollinger Band Signals
    signals['bb_signal'] = np.where(data['Close'] < signals['bb_lower'], 1,  # Price below lower band = Buy
                                  np.where(data['Close'] > signals['bb_upper'], -1, 0))  # Price above upper band = Sell
    
    # Combined Signal Strength
    signals['combined_signal'] = (
        signals['ma_signal'] + 
        signals['rsi_signal'] + 
        signals['macd_signal_flag'] + 
        signals['bb_signal']
    )
    
    # Volume-confirmed signals (only strong signals with above-average volume)
    signals['volume_confirmed'] = np.where(
        (abs(signals['combined_signal']) >= 2) & 
        (data['Volume'] > signals['volume_sma']), 
        signals['combined_signal'], 0
    )
    
    return signals

# ===============================
# LOAD DATA & ANALYZE
# ===============================

# Load your TCS data
data_path = r'C:\Users\rupan\newTry\financial-advisor\data\tcs_synthetic_5min.csv'
data = pd.read_csv(data_path, index_col=0, parse_dates=True)

# Generate complete analysis
signals = generate_trading_signals(data)

print("🚀 === COMPLETE TECHNICAL ANALYSIS DASHBOARD ===")
print(f"📊 Data Period: {data.index.min()} to {data.index.max()}")
print(f"📈 Total Candles: {len(data)}")
print()

# Current Market State
current = signals.iloc[-1]
current_price = data['Close'].iloc[-1]

print("💰 === CURRENT MARKET STATE ===")
print(f"Current Price: ₹{current_price:.2f}")
print(f"RSI: {current['rsi']:.1f} {'(OVERSOLD)' if current['rsi'] < 30 else '(OVERBOUGHT)' if current['rsi'] > 70 else '(NEUTRAL)'}")
print(f"MACD: {current['macd']:.3f}")
print(f"20-day SMA: ₹{current['sma_20']:.2f}")
print(f"50-day SMA: ₹{current['sma_50']:.2f}")
print()

# Recent Signals
print("🎯 === RECENT TRADING SIGNALS ===")
recent_signals = signals.tail(20)
strong_signals = recent_signals[abs(recent_signals['combined_signal']) >= 2]
volume_confirmed = recent_signals[recent_signals['volume_confirmed'] != 0]

print(f"Strong Signals (last 20 periods): {len(strong_signals)}")
print(f"Volume-Confirmed Signals: {len(volume_confirmed)}")

if len(volume_confirmed) > 0:
    print("\n🔥 LATEST VOLUME-CONFIRMED SIGNALS:")
    for idx, row in volume_confirmed.tail(3).iterrows():
        signal_type = "🟢 STRONG BUY" if row['volume_confirmed'] >= 2 else "🔴 STRONG SELL" if row['volume_confirmed'] <= -2 else "⚪ WEAK"
        print(f"{idx}: {signal_type} (Score: {row['volume_confirmed']})")

print()
print("📊 === INDICATOR SUMMARY ===")
print(f"RSI Trend: {'Bullish' if current['rsi'] > 50 else 'Bearish'}")
print(f"MA Trend: {'Bullish' if current['sma_20'] > current['sma_50'] else 'Bearish'}")
print(f"Volume Status: {'Above Average' if data['Volume'].iloc[-1] > current['volume_sma'] else 'Below Average'}")
print(f"Current Signal Score: {current['combined_signal']}")

print("\n✅ Technical Analysis Complete!")


🚀 === COMPLETE TECHNICAL ANALYSIS DASHBOARD ===
📊 Data Period: 2025-08-18 09:20:00 to 2025-09-12 15:25:00
📈 Total Candles: 1480

💰 === CURRENT MARKET STATE ===
Current Price: ₹3133.85
RSI: 37.0 (NEUTRAL)
MACD: -0.904
20-day SMA: ₹3134.85
50-day SMA: ₹3136.92

🎯 === RECENT TRADING SIGNALS ===
Strong Signals (last 20 periods): 1
Volume-Confirmed Signals: 1

🔥 LATEST VOLUME-CONFIRMED SIGNALS:
2025-09-12 14:30:00: 🟢 STRONG BUY (Score: 2)

📊 === INDICATOR SUMMARY ===
RSI Trend: Bearish
MA Trend: Bearish
Volume Status: Below Average
Current Signal Score: 0

✅ Technical Analysis Complete!


In [14]:
# === RISK MANAGEMENT & POSITION SIZING ===
def calculate_position_size(account_balance, risk_percent, entry_price, stop_loss_price):
    """Calculate position size based on risk management"""
    risk_amount = account_balance * (risk_percent / 100)
    price_risk = abs(entry_price - stop_loss_price)
    position_size = risk_amount / price_risk
    return position_size

def generate_trading_recommendations(data, signals, account_balance=100000):
    """Generate actionable trading recommendations"""
    recommendations = []
    
    # Get current values
    current_price = data['Close'].iloc[-1]
    current_rsi = signals['rsi'].iloc[-1]
    current_signal = signals['volume_confirmed'].iloc[-1]
    
    # Support and Resistance levels
    recent_high = data['High'].rolling(20).max().iloc[-1]
    recent_low = data['Low'].rolling(20).min().iloc[-1]
    
    if current_signal >= 2:  # Strong Buy Signal
        stop_loss = recent_low * 0.98  # 2% below recent low
        take_profit = current_price * 1.03  # 3% profit target
        position_size = calculate_position_size(account_balance, 2, current_price, stop_loss)
        
        recommendations.append({
            'action': 'BUY',
            'price': current_price,
            'quantity': int(position_size),
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'risk_reward': abs(take_profit - current_price) / abs(current_price - stop_loss),
            'confidence': 'HIGH'
        })
    
    elif current_signal <= -2:  # Strong Sell Signal
        stop_loss = recent_high * 1.02  # 2% above recent high
        take_profit = current_price * 0.97  # 3% profit target
        
        recommendations.append({
            'action': 'SELL',
            'price': current_price,
            'stop_loss': stop_loss,
            'take_profit': take_profit,
            'confidence': 'HIGH'
        })
    
    elif current_rsi < 30:  # RSI Oversold
        recommendations.append({
            'action': 'WATCH',
            'reason': 'RSI Oversold - Potential reversal setup',
            'entry_level': current_price * 0.999,
            'confidence': 'MEDIUM'
        })
    
    elif current_rsi > 70:  # RSI Overbought
        recommendations.append({
            'action': 'WATCH',
            'reason': 'RSI Overbought - Potential selling opportunity',
            'entry_level': current_price * 1.001,
            'confidence': 'MEDIUM'
        })
    
    else:
        recommendations.append({
            'action': 'HOLD',
            'reason': 'No strong signals - Wait for better setup',
            'confidence': 'LOW'
        })
    
    return recommendations

# Generate recommendations
recommendations = generate_trading_recommendations(data, signals)

print("\n💡 === TRADING RECOMMENDATIONS ===")
for i, rec in enumerate(recommendations, 1):
    print(f"\n🎯 Recommendation {i}:")
    print(f"Action: {rec['action']}")
    
    if 'price' in rec:
        print(f"Entry Price: ₹{rec['price']:.2f}")
    if 'quantity' in rec:
        print(f"Suggested Quantity: {rec['quantity']} shares")
    if 'stop_loss' in rec:
        print(f"Stop Loss: ₹{rec['stop_loss']:.2f}")
    if 'take_profit' in rec:
        print(f"Take Profit: ₹{rec['take_profit']:.2f}")
    if 'risk_reward' in rec:
        print(f"Risk:Reward Ratio: 1:{rec['risk_reward']:.2f}")
    if 'reason' in rec:
        print(f"Reason: {rec['reason']}")
    if 'entry_level' in rec:
        print(f"Entry Level: ₹{rec['entry_level']:.2f}")
    
    print(f"Confidence: {rec['confidence']}")

print(f"\n📈 === MARKET OUTLOOK ===")
if signals['rsi'].iloc[-1] < 40 and data['Close'].iloc[-1] < signals['bb_lower'].iloc[-1]:
    print("🟢 BULLISH BIAS: Multiple oversold conditions suggest potential upside")
elif signals['rsi'].iloc[-1] > 60 and data['Close'].iloc[-1] > signals['bb_upper'].iloc[-1]:
    print("🔴 BEARISH BIAS: Overbought conditions suggest potential downside")
else:
    print("⚪ NEUTRAL: Mixed signals - Wait for clearer direction")

print("\n⚠️ DISCLAIMER: This is for educational purposes only. Always do your own research!")



💡 === TRADING RECOMMENDATIONS ===

🎯 Recommendation 1:
Action: HOLD
Reason: No strong signals - Wait for better setup
Confidence: LOW

📈 === MARKET OUTLOOK ===
⚪ NEUTRAL: Mixed signals - Wait for clearer direction

⚠️ DISCLAIMER: This is for educational purposes only. Always do your own research!


In [15]:
# Save the complete system to a Python file
complete_system = '''
# TCS Technical Analysis System - Generated on Sept 12, 2025
# Your complete working trading system code goes here...
'''

with open(r'C:\Users\rupan\newTry\financial-advisor\complete_trading_system.py', 'w') as f:
    # Copy all your working code from the notebook to this file
    f.write("# Complete Trading System - Working Version\n")
    
print("✅ System ready to be saved as standalone script!")


✅ System ready to be saved as standalone script!


In [16]:
def analyze_multiple_timeframes(data):
    """Analyze trend across different timeframes"""
    print("🔍 === MULTI-TIMEFRAME ANALYSIS ===")
    
    # Short-term (20 periods)
    short_trend = "BULLISH" if data['Close'].rolling(20).mean().iloc[-1] > data['Close'].rolling(20).mean().iloc[-10] else "BEARISH"
    
    # Medium-term (50 periods)  
    medium_trend = "BULLISH" if data['Close'].rolling(50).mean().iloc[-1] > data['Close'].rolling(50).mean().iloc[-25] else "BEARISH"
    
    # Long-term (100 periods)
    long_trend = "BULLISH" if data['Close'].rolling(100).mean().iloc[-1] > data['Close'].rolling(100).mean().iloc[-50] else "BEARISH"
    
    print(f"📊 Short-term (1-hour): {short_trend}")
    print(f"📈 Medium-term (4-hour): {medium_trend}") 
    print(f"📉 Long-term (Daily): {long_trend}")
    
    # Trend Alignment Score
    bullish_count = sum([trend == "BULLISH" for trend in [short_trend, medium_trend, long_trend]])
    
    if bullish_count == 3:
        print("🚀 STRONG BULLISH ALIGNMENT - All timeframes bullish!")
    elif bullish_count == 0:
        print("🔴 STRONG BEARISH ALIGNMENT - All timeframes bearish!")
    else:
        print("⚪ MIXED SIGNALS - Conflicting timeframes")
    
    return {'short': short_trend, 'medium': medium_trend, 'long': long_trend}

# Run multi-timeframe analysis
trends = analyze_multiple_timeframes(data)


🔍 === MULTI-TIMEFRAME ANALYSIS ===
📊 Short-term (1-hour): BEARISH
📈 Medium-term (4-hour): BEARISH
📉 Long-term (Daily): BULLISH
⚪ MIXED SIGNALS - Conflicting timeframes


In [17]:
def detect_market_regime(data, signals):
    """Detect if market is trending, ranging, or volatile"""
    
    # Calculate volatility (20-period standard deviation)
    volatility = data['Close'].rolling(20).std()
    current_vol = volatility.iloc[-1]
    avg_vol = volatility.mean()
    
    # Calculate trend strength
    price_change = (data['Close'].iloc[-1] - data['Close'].iloc[-20]) / data['Close'].iloc[-20] * 100
    
    # Bollinger Band width (measure of volatility)
    bb_width = (signals['bb_upper'].iloc[-1] - signals['bb_lower'].iloc[-1]) / signals['bb_upper'].iloc[-1] * 100
    
    print("\n🌡️ === MARKET REGIME ANALYSIS ===")
    print(f"Current Volatility: {current_vol:.2f}")
    print(f"Average Volatility: {avg_vol:.2f}")
    print(f"20-period Return: {price_change:.2f}%")
    print(f"Bollinger Band Width: {bb_width:.2f}%")
    
    # Determine regime
    if abs(price_change) > 2 and current_vol > avg_vol * 1.2:
        regime = "TRENDING"
        strategy = "Follow momentum, use trend-following indicators"
    elif bb_width < 2 and abs(price_change) < 1:
        regime = "RANGING"
        strategy = "Use mean reversion, buy at support/sell at resistance"
    elif current_vol > avg_vol * 1.5:
        regime = "VOLATILE"
        strategy = "Reduce position size, wait for stability"
    else:
        regime = "NEUTRAL"
        strategy = "Mixed approach, wait for clearer signals"
    
    print(f"\n📊 Market Regime: {regime}")
    print(f"💡 Suggested Strategy: {strategy}")
    
    return regime

regime = detect_market_regime(data, signals)



🌡️ === MARKET REGIME ANALYSIS ===
Current Volatility: 0.87
Average Volatility: 3.29
20-period Return: -0.06%
Bollinger Band Width: 0.11%

📊 Market Regime: RANGING
💡 Suggested Strategy: Use mean reversion, buy at support/sell at resistance


In [18]:
def risk_assessment(data, signals):
    """Comprehensive risk analysis"""
    print("\n⚠️ === RISK ASSESSMENT ===")
    
    # Recent drawdown
    rolling_max = data['Close'].rolling(50).max()
    drawdown = (data['Close'] - rolling_max) / rolling_max * 100
    max_drawdown = drawdown.min()
    current_drawdown = drawdown.iloc[-1]
    
    # Volatility percentile
    vol_20 = data['Close'].rolling(20).std()
    vol_percentile = (vol_20.iloc[-1] > vol_20.quantile(0.8))
    
    # RSI divergence check
    recent_rsi = signals['rsi'].tail(10)
    recent_prices = data['Close'].tail(10)
    
    rsi_trend = "RISING" if recent_rsi.iloc[-1] > recent_rsi.iloc[-5] else "FALLING"
    price_trend = "RISING" if recent_prices.iloc[-1] > recent_prices.iloc[-5] else "FALLING"
    
    divergence = (rsi_trend != price_trend)
    
    print(f"📉 Max Drawdown (50-period): {max_drawdown:.2f}%")
    print(f"📊 Current Drawdown: {current_drawdown:.2f}%")
    print(f"🌡️ High Volatility Alert: {'YES' if vol_percentile else 'NO'}")
    print(f"⚡ RSI-Price Divergence: {'YES' if divergence else 'NO'}")
    
    # Risk Score (0-10)
    risk_score = 0
    if abs(max_drawdown) > 5: risk_score += 3
    if vol_percentile: risk_score += 2
    if divergence: risk_score += 2
    if abs(current_drawdown) > 3: risk_score += 3
    
    risk_level = "LOW" if risk_score <= 3 else "MEDIUM" if risk_score <= 6 else "HIGH"
    
    print(f"\n🎯 Overall Risk Score: {risk_score}/10 ({risk_level})")
    
    if risk_level == "HIGH":
        print("🚨 HIGH RISK: Consider reducing position size or staying in cash")
    elif risk_level == "MEDIUM":
        print("⚠️ MEDIUM RISK: Trade with caution, use tight stops")
    else:
        print("✅ LOW RISK: Normal trading conditions")

risk_assessment(data, signals)



⚠️ === RISK ASSESSMENT ===
📉 Max Drawdown (50-period): -1.47%
📊 Current Drawdown: -0.21%
🌡️ High Volatility Alert: NO
⚡ RSI-Price Divergence: NO

🎯 Overall Risk Score: 0/10 (LOW)
✅ LOW RISK: Normal trading conditions


In [19]:
def track_signal_performance(signals):
    """Track how well your signals have performed"""
    print("\n📈 === SIGNAL PERFORMANCE TRACKING ===")
    
    # Find all strong signals
    strong_signals = signals[abs(signals['volume_confirmed']) >= 2].copy()
    
    if len(strong_signals) > 0:
        print(f"Total Strong Signals Generated: {len(strong_signals)}")
        print(f"Buy Signals: {len(strong_signals[strong_signals['volume_confirmed'] > 0])}")
        print(f"Sell Signals: {len(strong_signals[strong_signals['volume_confirmed'] < 0])}")
        
        # Show recent strong signals
        print("\n🔥 Recent Strong Signals:")
        for idx, row in strong_signals.tail(5).iterrows():
            signal_type = "BUY" if row['volume_confirmed'] > 0 else "SELL"
            print(f"{idx}: {signal_type} (Score: {row['volume_confirmed']})")
    else:
        print("No strong signals generated in this period")
        print("✅ System is being conservative - good risk management!")

track_signal_performance(signals)



📈 === SIGNAL PERFORMANCE TRACKING ===
Total Strong Signals Generated: 32
Buy Signals: 16
Sell Signals: 16

🔥 Recent Strong Signals:
2025-09-09 09:30:00: SELL (Score: -2)
2025-09-10 12:15:00: SELL (Score: -2)
2025-09-10 13:00:00: SELL (Score: -2)
2025-09-11 10:15:00: SELL (Score: -2)
2025-09-12 14:30:00: BUY (Score: 2)


In [20]:
# Complete enhanced analysis
print("=" * 60)
print("🚀 ENHANCED TCS TECHNICAL ANALYSIS SYSTEM")
print("=" * 60)

analyze_multiple_timeframes(data)
detect_market_regime(data, signals)
risk_assessment(data, signals)
track_signal_performance(signals)

print("\n" + "=" * 60)
print("✅ ENHANCED ANALYSIS COMPLETE!")
print("=" * 60)


🚀 ENHANCED TCS TECHNICAL ANALYSIS SYSTEM
🔍 === MULTI-TIMEFRAME ANALYSIS ===
📊 Short-term (1-hour): BEARISH
📈 Medium-term (4-hour): BEARISH
📉 Long-term (Daily): BULLISH
⚪ MIXED SIGNALS - Conflicting timeframes

🌡️ === MARKET REGIME ANALYSIS ===
Current Volatility: 0.87
Average Volatility: 3.29
20-period Return: -0.06%
Bollinger Band Width: 0.11%

📊 Market Regime: RANGING
💡 Suggested Strategy: Use mean reversion, buy at support/sell at resistance

⚠️ === RISK ASSESSMENT ===
📉 Max Drawdown (50-period): -1.47%
📊 Current Drawdown: -0.21%
🌡️ High Volatility Alert: NO
⚡ RSI-Price Divergence: NO

🎯 Overall Risk Score: 0/10 (LOW)
✅ LOW RISK: Normal trading conditions

📈 === SIGNAL PERFORMANCE TRACKING ===
Total Strong Signals Generated: 32
Buy Signals: 16
Sell Signals: 16

🔥 Recent Strong Signals:
2025-09-09 09:30:00: SELL (Score: -2)
2025-09-10 12:15:00: SELL (Score: -2)
2025-09-10 13:00:00: SELL (Score: -2)
2025-09-11 10:15:00: SELL (Score: -2)
2025-09-12 14:30:00: BUY (Score: 2)

✅ ENHANCED 